In [6]:
result = {'barcode': '2500000279935', 'cert_mark': ['organic'], 'cert_result': {'12100489': {'in_db': True, 'mark_status': 'success', 'name': {'db': '김영대', 'ocr_rslt': '김영대', 'score': 1.0}, 'number': {'db': '12100489', 'ocr_rslt': '12100409', 'score': 0.875}}}, 'date_time': '02/15/2022, 15:35:24', 'label_id': 157, 'label_loc': '/mnt/vitasoft/salim/detected_labels/2500000279935/0157.png', 'product_name': {'name': '친환경 방울토마토', 'status': 'success'}, 'rot_angle': 180, 'weight': {'db': '600g', 'ocr_rslt': '600g', 'score': 1.0, 'status': 'success'}}

result1 = {'barcode': '2500000279935', 'cert_mark': ['organic'], 'cert_result': {'04829818': {'in_db': False, 'mark_status': 'success', 'number': {'db': '04829818', 'ocr_rslt': '04829818'}}, '12100489': {'in_db': True, 'mark_status': 'success', 'name': {'db': '김영대', 'ocr_rslt': '김영대', 'score': 1.0}, 'number': {'db': '12100489', 'ocr_rslt': '12100489', 'score': 1.0}}}, 'date_time': '02/15/2022, 16:40:53', 'label_id': 562, 'label_loc': '/mnt/vitasoft/salim/detected_labels/2500000279935/0562.png', 'product_name': {'name': '친환경 방울토마토', 'status': 'success'}, 'rot_angle': 0, 'weight': {'db': '600g', 'ocr_rslt': '600g', 'score': 1.0, 'status': 'success'}}

result2 = {'barcode': '2500000145629', 'cert_mark': ['organic'], 'cert_result': {}, 'date_time': '02/17/2022, 15:53:04', 'label_id': 142, 'label_loc': '/mnt/vitasoft/salim/detected_labels/2500000145629/0142.png', 'product_name': {'name': '유기농 표고버섯', 'status': 'success'}, 'rot_angle': 90, 'weight': {'db': '300g', 'ocr_rslt': '300g', 'score': 1.0, 'status': 'success'}}

mark_kor = {
    'organic': '유기농',
    'nonpesticide': '무농약',
    'gap': 'GAP(우수관리인증)',
    'antibiotic': '무항생제',
    'animal': '동물복지',
    'haccp': '안전관리인증HACCP',
    'pgi': '지리적표시',
    'traditional': '전통식품',
    'master': '식품명인',
    'processed': '가공식품',
    'carbon': '저탄소(LOW CARBON)'
}

def inspection_result(result):
    return_dict = {'product_name': [], 'weight': [], 'barcode': [], 'cert_mark': [], 'cert_result': []}
    for key, value in result.items():
        cert_third_num = []
        if key == 'product_name':
            return_dict[key].append(value['name'])
            return_dict[key].append(value['name'])
            if value['status'] == 'success':
                return_dict[key].append('100%')
                return_dict[key].append('승인')

        if key == 'weight':
            return_dict[key].append(value['ocr_rslt'])
            return_dict[key].append(value['db'])
            return_dict[key].append(f"{round(value['score'] * 100)}%")
            return_dict[key].append('승인')

        if key == 'barcode':
            return_dict[key].append(value)
            return_dict[key].append(value)
            return_dict[key].append('100%')
            return_dict[key].append('승인')

        if key == 'cert_mark':
            for mark in value:
                mark_lst = []
                mark_lst.append(mark_kor[mark])
                cert_numbers = [item['number']['ocr_rslt'] for item in result['cert_result'].values()]
                mark_lst.append(', '.join([number[2] for number in cert_numbers]))
                mark_status = [number['mark_status'] for number in result['cert_result'].values()]
                count = 0
                for status in mark_status:
                    if 'success' == status:
                        count += 1
                if len(mark_status) > 0:
                    raw_score = count / len(mark_status)
                else:
                    raw_score = 0.0
                score = f"{round(raw_score * 100)}%"
                mark_lst.append(score)

                if count == 2 and raw_score >= 0.9:
                    mark_lst.append('승인')
                elif 'fail' in mark_status:
                    mark_lst.append('오류')
                else:
                    mark_lst.append('매칭실패')

                return_dict[key].append(mark_lst)

        if key == 'cert_result':
            for num_key, num_value in value.items():
                cert_lst = []
                if num_value['in_db']:
                    cert_lst.append(f"{num_value['number']['ocr_rslt']} {num_value['name']['ocr_rslt']}")
                    cert_lst.append(f"{num_value['number']['db']} {num_value['name']['db']}")
                    raw_score = (num_value['name']['score'] + num_value['number']['score']) / 2.0
                    cert_lst.append(f"{round(raw_score * 100)}%")
                    result = '오류'
                    if raw_score >= 0.9:
                        result = '승인'
                    cert_lst.append(result)
                else:
                    name = ''
                    number = ''
                    if 'name' in num_value and 'number' in num_value:
                        name = num_value['name']['ocr_rslt']
                        number = num_value['number']['ocr_rslt']
                    elif 'number' in num_value.keys():
                        number = num_value['number']['ocr_rslt']
                    elif 'name' in num_value.keys():
                        name = num_value['name']['ocr_rslt']
                    cert_lst.append(f"{name} {number}")
                    cert_lst.append("매칭실패")
                    cert_lst.append('0%')
                    cert_lst.append('매칭실패')
                return_dict[key].append(cert_lst)

    return return_dict

In [23]:
from glob import glob
import os
result_dict = inspection_result(result2)
print(result_dict['product_name'][1])
root_path = 'A:/salim/detected_labels'
len(glob(os.path.join(root_path, str(result_dict['barcode'][0]), '*.png')))

유기농 표고버섯


255

In [40]:
import pandas as pd
class Db():
    def __init__(self):
        self.df = pd.read_excel('../../Database/제품등록정보20211015.xlsx')

    def get_product_names(self):
        lst = ['라벨제품명', '라벨제품명별칭']
        product_names = list(self.df["라벨제품명"])
        product_names.remove('운영중단')
        return product_names

db = Db()
db.get_product_names()

['유기농 깐 밤',
 '유기농 표고버섯',
 '무농약완숙토마토',
 '친환경 캠벨 포도',
 '친환경 토마토',
 '친환경 백오이',
 '저탄소인증 샤인머스켓',
 '친환경 브로커리',
 '친환경 애호박',
 '유기농 새송이버섯',
 '유기농 팽이버섯',
 '친환경 대추방울토마토',
 '유기농 표고버섯',
 '친환경 방울토마토',
 '친환경 대추방울토마토',
 '무농약 갈색양송이',
 '에브리데이 방울토마토',
 '에브리데이 토마토',
 '국산 유기농 바나나',
 '친환경 토마토',
 '친환경 무화과',
 '저탄소 레드키위']